In [2]:
import sys
sys.path.insert(0, '../')

In [3]:
from pgso.test_functions import *
from pgso.gso import GSO as PGSO, PSO_purana
from pgso.benchmark import *
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import scipy.io
from numba import jit

# PSO IMPLEMENTATION

In [4]:
#dependencies
import random
import math
import copy # for array copying
import sys

class Particle:
    def __init__(self,x0, num_dimensions):
        self.position_i=[]          # particle position
        self.velocity_i=[]          # particle velocity
        self.pos_best_i=[]          # best position individual
        self.err_best_i=-1          # best error individual
        self.err_i=-1               # error individual
        self.num_dimensions = num_dimensions
        
        for i in range(0, self.num_dimensions):
            self.velocity_i.append(random.uniform(-1,1))
            self.position_i.append(x0[i])

    # evaluate current fitness
    def evaluate(self,costFunc):
        self.err_i=costFunc(self.position_i)

        # check to see if the current position is an individual best
        if self.err_i < self.err_best_i or self.err_best_i==-1:
            self.pos_best_i=self.position_i
            self.err_best_i=self.err_i

    # update new particle velocity
    def update_velocity(self,pos_best_g):
        w=0.5       # constant inertia weight (how much to weigh the previous velocity)
        c1=1        # cognative constant
        c2=2        # social constant

        for i in range(0, self.num_dimensions):
            r1=random.random()
            r2=random.random()

            vel_cognitive=c1*r1*(self.pos_best_i[i]-self.position_i[i])
            vel_social=c2*r2*(pos_best_g[i]-self.position_i[i])
            self.velocity_i[i]=w*self.velocity_i[i]+vel_cognitive+vel_social

    # update the particle position based off new velocity updates
    def update_position(self,bounds):
        for i in range(0, self.num_dimensions):
            self.position_i[i]=self.position_i[i]+self.velocity_i[i]

            # adjust maximum position if necessary
            if self.position_i[i]>bounds[i][1]:
                self.position_i[i]=bounds[i][1]

            # adjust minimum position if neseccary
            if self.position_i[i] < bounds[i][0]:
                self.position_i[i]=bounds[i][0]

def PSO(costFunc,bounds,maxiter, swarm_init):
    num_dimensions=len(swarm_init[0])
    err_best_g=-1                   # best error for group
    pos_best_g=[]                   # best position for group
    num_particles = len(swarm_init)
    # establish the swarm
    swarm = [Particle(position, num_dimensions) for position in swarm_init]
    # begin optimization loop
    i=0
    while i < maxiter:
        #print i,err_best_g
        # cycle through particles in swarm and evaluate fitness
        for j in range(0,num_particles):
            swarm[j].evaluate(costFunc)

            # determine if current particle is the best (globally)
            if swarm[j].err_i < err_best_g or err_best_g == -1:
                pos_best_g=list(swarm[j].position_i)
                err_best_g=float(swarm[j].err_i)

        # cycle through swarm and update velocities and position
        for j in range(0,num_particles):
            swarm[j].update_velocity(pos_best_g)
            swarm[j].update_position(bounds)
        i+=1
    return pos_best_g, err_best_g

# GSO IMPLEMENTATION

In [5]:
def GSO(M, bounds, num_particles, max_iter, costfunc):
    subswarm_bests = []
    dims = len(bounds)
    lb = bounds[0][0] 
    ub = bounds[0][1] 
    for i in range(M):
        swarm_init = [np.random.uniform(lb, ub, dims) for _ in range(num_particles)]
        subswarm_best,_ = PSO(costfunc,bounds,max_iter, swarm_init=swarm_init)
        subswarm_bests.append(subswarm_best)
    best_position, best_error = PSO(costfunc, bounds, max_iter, swarm_init=subswarm_bests)
    return best_position, best_error

# ROTATED and SHIFTED FUNCTIONS

In [6]:
def rotated_rastrigin(x):
    if len(x) == 10:
        mat = scipy.io.loadmat('./matlab-files/rastrigin_M_D10.mat')
    elif len(x) == 30:
        mat = scipy.io.loadmat('./matlab-files/rastrigin_M_D30.mat')
    elif len(x) == 50:
        mat = scipy.io.loadmat('./matlab-files/rastrigin_M_D50.mat')
    y = np.matmul(mat['M'],x)
    return rastrigin(y)

def rotated_griewangk(x):
    if len(x) == 10:
        mat = scipy.io.loadmat('./matlab-files/griewank_M_D10.mat')
    elif len(x) == 30:
        mat = scipy.io.loadmat('./matlab-files/griewank_M_D30.mat')
    elif len(x) == 50:
        mat = scipy.io.loadmat('./matlab-files/griewank_M_D50.mat')
    y = np.matmul(mat['M'],x)
    return griewank(y)

def rotated_ackley(x):
    if len(x) == 10:
        mat = scipy.io.loadmat('./matlab-files/ackley_M_D10.mat')
    elif len(x) == 30:
        mat = scipy.io.loadmat('./matlab-files/ackley_M_D30.mat')
    elif len(x) == 50:
        mat = scipy.io.loadmat('./matlab-files/ackley_M_D50.mat')
    y = np.matmul(mat['M'],x)
    return ackley(x)

def shifted_rotated_rastrigin(x):
    o = np.random.uniform(-2, 2, len(x))
    x = x - o
    return rotated_rastrigin(x)

def shifted_rotated_ackley(x):
    o = np.random.uniform(-2, 2, len(x))
    x = x - o
    return rotated_ackley(x)

In [7]:
unimodal_functions = [exponential, powellsumfcn, sum_of_squares, schfewel_220, schwefel_222, griewank, zakharov, sphere]
unimodal_strings = ['exponential', ' powell sum function', ' sum_of_squares', ' schfewel 2.20', ' schwefel 2.22', ' griewank', ' zakharov', ' sphere']
unimodal_bounds = [[-1, 1], [-1, 1], [-10, 10], [-100, 100], [-100, 100], [-600, 600], [-5, 10], [-100, 100]]

multimodal_functions = [nonContinuousRastrigin, ackley, rastrigin, rosen, rotated_rastrigin, rotated_griewangk, rotated_ackley, shifted_rotated_rastrigin, shifted_rotated_ackley]
multimodal_strings = ['nonContinuousRastrigin', 'ackley', 'rastrigin', 'rosen', "rotated_rastrigin", "rotated_griewangk", "rotated_ackley", "shifted_rotated_rastrigin", "shifted_rotated_ackley"]
multimodal_bounds = [[-100, 100], [-40, 40], [-100, 100], [-30, 30], [-100, 100], [-600, 600], [-40, 40], [-5.12, 5.12], [-10, 10]]

In [8]:
def get_GSO_results(dimensions, bounds, costfunc, algorithm, M, num_particles, max_iter, suppress=True):
    search_space = [bounds for _ in range(dimensions)]
    if not suppress:
        print("\n Algorithm: ", algorithm,"\n Dimensions: ", dimensions,"\n cost function: ", costfunc,"\n iterations: ", max_iter)
    score = 0
    for _ in range(10):
        score += algorithm(M, search_space, num_particles, max_iter, costfunc)[1]
    score = score / 10
    return score

def run_test(dimensions, algorithm, M, num_particles, max_iter, mode="unimodal"):
    modal_tests = dict()
    if mode == "unimodal":
        for func, bnds, stri in zip(unimodal_functions, unimodal_bounds, unimodal_strings):
            modal_tests[stri] = get_GSO_results(dimensions, bnds, func, algorithm, M, num_particles, max_iter)
    else:
        for func, bnds, stri in zip(multimodal_functions, multimodal_bounds, multimodal_strings):
            modal_tests[stri] = get_GSO_results(dimensions, bnds, func, algorithm, M, num_particles, max_iter)
    return modal_tests

# Unimodal on GSO

In [35]:
print(run_test(10, GSO, 7, 20, 1000))
print(run_test(30, GSO, 7, 20, 1000))
print(run_test(50, GSO, 7, 20, 1000))

{'exponential': -0.9999999999999944, ' powell sum function': 3.5758827127586123e-09, ' sum_of_squares': 4.173261881599082e-15, ' schfewel 2.20': 0.03901460767474148, ' schwefel 2.22': 60.89823430429923, ' griewank': 0.11660587138828551, ' zakharov': 0.01933171683124276, ' sphere': 4.4296815948813486e-15}
{'exponential': -0.8357040003081604, ' powell sum function': 4.847038868571392e-05, ' sum_of_squares': 166.89183812312427, ' schfewel 2.20': 256.3515965692038, ' schwefel 2.22': 1004.8097915925216, ' griewank': 23.167782131831665, ' zakharov': 419.7685212711157, ' sphere': 506.0477587739568}
{'exponential': -0.4158082289302059, ' powell sum function': 0.000496260844818537, ' sum_of_squares': 3132.5181717458045, ' schfewel 2.20': 730.4727024445715, ' schwefel 2.22': 1899.7863700748214, ' griewank': 95.32994853505573, ' zakharov': 1008.788011052161, ' sphere': 9851.406296567588}


# Multimodal On GSO

In [38]:
print(run_test(10, GSO, 7, 20, 1000, "multimodal"))
print(run_test(30, GSO, 7, 20, 1000, "multimodal"))
print(run_test(50, GSO, 7, 20, 1000, "multimodal"))

{'nonContinuousRastrigin': 28.405503697535302, 'ackley': 1.6874087146866756, 'rastrigin': 32.63602528462998, 'rosen': 5.200615042737145, 'rotated_rastrigin': 46.813335445181316, 'rotated_griewangk': 0.32503702657477496, 'rotated_ackley': 1.9517829210475557, 'shifted_rotated_rastrigin': 87.05403437132182, 'shifted_rotated_ackley': 4.404317307954242}
{'nonContinuousRastrigin': 1661.5908171298029, 'ackley': 19.52829931415535, 'rastrigin': 1099.4411045178742, 'rosen': 32716.237738523225, 'rotated_rastrigin': 11291.556480095092, 'rotated_griewangk': 162.49799283670683, 'rotated_ackley': 19.3806081600923, 'shifted_rotated_rastrigin': 473.65034908929954, 'shifted_rotated_ackley': 10.826823909590434}
{'nonContinuousRastrigin': 18378.96001992069, 'ackley': 19.975863094473095, 'rastrigin': 18410.53006056455, 'rosen': 2315366.1817619773, 'rotated_rastrigin': 45400.88049215202, 'rotated_griewangk': 883.5469045765418, 'rotated_ackley': 19.93253433234317, 'shifted_rotated_rastrigin': 982.56569464534

# Unimodal on PGSO

In [36]:
print(run_test(10, PGSO, 7, 20, 1000))
print(run_test(30, PGSO, 7, 20, 1000))
print(run_test(50, PGSO, 7, 20, 1000))

{'exponential': 0.0, ' powell sum function': 0.0, ' sum_of_squares': 0.0, ' schfewel 2.20': 0.0, ' schwefel 2.22': 0.0, ' griewank': 0.0, ' zakharov': 0.0, ' sphere': 0.0}
{'exponential': 0.0, ' powell sum function': 0.0, ' sum_of_squares': 0.0, ' schfewel 2.20': 0.0, ' schwefel 2.22': 0.0, ' griewank': 0.09949590570932898, ' zakharov': 0.0, ' sphere': 0.0}
{'exponential': 0.0, ' powell sum function': 0.0, ' sum_of_squares': 0.0, ' schfewel 2.20': 0.0, ' schwefel 2.22': 0.0, ' griewank': 0.09949590570932898, ' zakharov': 0.0, ' sphere': 0.0}


# Multimodal on PGSO

In [39]:
print(run_test(10, PGSO, 7, 20, 1000, "multimodal"))
print(run_test(30, PGSO, 7, 20, 1000, "multimodal"))
print(run_test(50, PGSO, 7, 20, 1000, "multimodal"))

{'nonContinuousRastrigin': 0.0, 'ackley': 0.0, 'rastrigin': 0.0, 'rosen': 0.19899181141865796, 'rotated_rastrigin': 0.19899181141865796, 'rotated_griewangk': 0.09949590570932898, 'rotated_ackley': 0.0, 'shifted_rotated_rastrigin': 0.09949590570932898, 'shifted_rotated_ackley': 0.19899181141865796}
{'nonContinuousRastrigin': 0.0, 'ackley': 0.0, 'rastrigin': 0.0, 'rosen': 0.09949590570932898, 'rotated_rastrigin': 0.0, 'rotated_griewangk': 0.0, 'rotated_ackley': 0.09949590570932898, 'shifted_rotated_rastrigin': 0.19899181141865796, 'shifted_rotated_ackley': 0.0}
{'nonContinuousRastrigin': 0.0, 'ackley': 0.0, 'rastrigin': 0.0, 'rosen': 0.09949590570932898, 'rotated_rastrigin': 0.0, 'rotated_griewangk': 0.0, 'rotated_ackley': 0.0, 'shifted_rotated_rastrigin': 0.19899181141865796, 'shifted_rotated_ackley': 0.0}


# PSO

In [9]:
def True_PSO(costFunc,bounds,maxiter, num_particles):
    lb = bounds[0][0]
    ub = bounds[0][1]
    
    num_dimensions=len(bounds)
    swarm_init = np.array([np.random.uniform(lb, ub, num_dimensions) for _ in range(num_particles)])
    
    err_best_g=-1                   # best error for group
    pos_best_g=[]                   # best position for group
    # establish the swarm
    swarm = [Particle(position, num_dimensions) for position in swarm_init]
    # begin optimization loop
    i=0
    while i < maxiter:
        #print i,err_best_g
        # cycle through particles in swarm and evaluate fitness
        for j in range(0,num_particles):
            swarm[j].evaluate(costFunc)

            # determine if current particle is the best (globally)
            if swarm[j].err_i < err_best_g or err_best_g == -1:
                pos_best_g=list(swarm[j].position_i)
                err_best_g=float(swarm[j].err_i)

        # cycle through swarm and update velocities and position
        for j in range(0,num_particles):
            swarm[j].update_velocity(pos_best_g)
            swarm[j].update_position(bounds)
        i+=1
    return pos_best_g, err_best_g

In [10]:
def run_PSO_tests(dimensions, maxiter, num_particles, mode="unimodal"):
    results_dict = dict()
    if mode == "unimodal":
        functions = unimodal_functions
        strings = unimodal_strings
        bounds = unimodal_bounds
    else:
        functions = multimodal_functions
        strings = multimodal_strings
        bounds = multimodal_bounds

    for func, bnds, stri in zip(functions, bounds, strings):
        search_space = [bnds for _ in range(dimensions)]            
        score = 0
        for _ in range(10):
            score += True_PSO(func, search_space, maxiter, num_particles)[1]
        score = score/10
        results_dict[stri] = score
    return results_dict    

# Unimodal on PSO

In [21]:
print(run_PSO_tests(10, 1000, 50))
print(run_PSO_tests(30, 1000, 50))
print(run_PSO_tests(50, 1000, 50))

{'exponential': -0.9606530659712632, ' powell sum function': 1.294820677780308e-14, ' sum_of_squares': 40.00000000000001, ' schfewel 2.20': 10.028387353541428, ' schwefel 2.22': 218.96078690592313, ' griewank': 0.2135347715875347, ' zakharov': 54.407368891183594, ' sphere': 1.8000001596299296e-85}
{'exponential': -0.569954966783031, ' powell sum function': 0.20000094266331553, ' sum_of_squares': 1978.1932797921913, ' schfewel 2.20': 322.7206393196454, ' schwefel 2.22': 1122.0745010542857, ' griewank': 129.3991502466623, ' zakharov': 569.5838732189188, ' sphere': 10191.371524269203}
{'exponential': -0.2046134408101108, ' powell sum function': 0.10003236426881243, ' sum_of_squares': 9268.900390238367, ' schfewel 2.20': 874.4493281543882, ' schwefel 2.22': 1996.607858206618, ' griewank': 353.09732788997127, ' zakharov': 1389.0649821095462, ' sphere': 27710.16735336067}


# Multimodal on PSO

In [ ]:
print(run_PSO_tests(10, 1000, 50, 'multimodal'))
print(run_PSO_tests(30, 1000, 50, 'multimodal'))
print(run_PSO_tests(50, 1000, 50, 'multimodal'))